To import the raw data in FITS format, I will use the [Astropy](https://www.astropy.org) package.

In [5]:
# pip install astropy
from astropy.io import fits

In [6]:
import os
import requests

base_url = "https://umbra.nascom.nasa.gov/goes/fits"

- As we will see, importing even one year of data is challenging. 
- For this example I will focus on the year 2018.

In [63]:
import pandas as pd
import numpy as np

records = []

for year in range(2005, 2021):
    df = pd.read_html(f"{base_url}/{year}")[0][["Name"]].dropna()
    files = df["Name"][df["Name"].str.endswith("fits")]
    for file in files:
        month = int(file[:-5][-4:-2])
        # if month != 1: continue
        day = int(file[:-5][-2:])
        outfile = "fits/" + file
        if not os.path.exists(outfile):
            urllib.request.urlretrieve(f"{base_url}/{year}/{file}", outfile)
            print(file)
        with fits.open(outfile) as io:
            fld = np.asarray(io["FLUXES"].data)
            # The time unit is seconds from midnight UTC, observations
            # are approximately once every two seconds
            if "TIME" in fld.dtype.names:
                tim = fld["TIME"][0,:]
            # The flux levels in watts per square meter
                flx1, flx2 = fld["FLUX"][0,:,:].T
            else:                
                tim = fld["Time"][0,:]
                # The flux levels in watts per square meter
                flx1, flx2 = fld["Flux"][0,:,:].T
            records.append({
                'year': year, 'month': month, 'day': day, 'sat': file[:4],
                'flux1': flx1[flx1>0].mean(), 'flux2': flx2[flx2>0].mean()
            })

/tmp/ipykernel_1983772/2824728063.py:31: RuntimeWarning: Mean of empty slice.
  'flux1': flx1[flx1>0].mean(), 'flux2': flx2[flx2>0].mean()
/home/terhorst/opt/jax_gpu/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_1983772/2824728063.py:31: RuntimeWarning: Mean of empty slice.
  'flux1': flx1[flx1>0].mean(), 'flux2': flx2[flx2>0].mean()
/home/terhorst/opt/jax_gpu/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_1983772/2824728063.py:31: RuntimeWarning: Mean of empty slice.
  'flux1': flx1[flx1>0].mean(), 'flux2': flx2[flx2>0].mean()
/home/terhorst/opt/jax_gpu/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_1983772/2824728063.py:31: RuntimeWarning: Mean of

go1420190101.fits
go1420190102.fits
go1420190103.fits
go1420190104.fits
go1420190105.fits
go1420190106.fits
go1420190107.fits
go1420190108.fits
go1420190109.fits
go1420190110.fits
go1420190111.fits
go1420190112.fits
go1420190113.fits
go1420190114.fits
go1420190115.fits
go1420190116.fits
go1420190117.fits
go1420190118.fits
go1420190119.fits
go1420190120.fits
go1420190121.fits
go1420190122.fits
go1420190123.fits
go1420190124.fits
go1420190125.fits
go1420190126.fits
go1420190127.fits
go1420190128.fits
go1420190129.fits
go1420190130.fits
go1420190131.fits
go1420190201.fits
go1420190202.fits
go1420190203.fits
go1420190204.fits
go1420190205.fits
go1420190206.fits
go1420190207.fits
go1420190208.fits
go1420190209.fits
go1420190210.fits
go1420190211.fits
go1420190212.fits
go1420190213.fits
go1420190214.fits
go1420190215.fits
go1420190216.fits
go1420190217.fits
go1420190218.fits
go1420190219.fits
go1420190220.fits
go1420190221.fits
go1420190222.fits
go1420190223.fits
go1420190224.fits
go14201902

go1520190407.fits
go1520190408.fits
go1520190409.fits
go1520190410.fits
go1520190411.fits
go1520190412.fits
go1520190413.fits
go1520190414.fits
go1520190415.fits
go1520190416.fits
go1520190417.fits
go1520190418.fits
go1520190419.fits
go1520190420.fits
go1520190421.fits
go1520190422.fits
go1520190423.fits
go1520190424.fits
go1520190425.fits
go1520190426.fits
go1520190427.fits
go1520190428.fits
go1520190429.fits
go1520190430.fits
go1520190501.fits
go1520190502.fits
go1520190503.fits
go1520190504.fits
go1520190505.fits
go1520190506.fits
go1520190507.fits
go1520190508.fits
go1520190509.fits
go1520190510.fits
go1520190511.fits
go1520190512.fits
go1520190513.fits
go1520190514.fits
go1520190515.fits
go1520190516.fits
go1520190517.fits
go1520190518.fits
go1520190519.fits
go1520190520.fits
go1520190521.fits
go1520190522.fits
go1520190523.fits
go1520190524.fits
go1520190525.fits
go1520190526.fits
go1520190527.fits
go1520190528.fits
go1520190529.fits
go1520190530.fits
go1520190531.fits
go15201906

In [64]:
df = pd.DataFrame.from_records(records)

In [65]:
df["sat"] = df["sat"].astype('category')

In [66]:
import pandas as pd
import pyarrow as pa
import pyarrow.feather as feather

In [67]:
table = pa.Table.from_pandas(df)

# Save to Feather file
feather.write_feather(table, 'goes_daily.feather')